# Neural Network From Scratch (NumPy)

A compact, self-contained walkthrough of building and training a tiny neural network using **only NumPy**.
It includes forward pass, backpropagation, and gradient descent updates, plus a few demo tasks.


## Setup
We use NumPy for vectorized math and Matplotlib for quick visual checks.


In [ ]:
import numpy as np
np.set_printoptions(suppress=True)  # avoid scientific notation in prints
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # required for 3D plots

np.random.seed(10)


## Small helpers
- `add_dimension_optional` ensures consistent shapes.
- `visualize_loss` plots training loss.
- `visualize_preds` compares a learned surface vs the real function.
- `get_accuracy` computes binary accuracy (threshold at 0.5).


In [ ]:
def add_dimension_optional(arr):
    new_arr = arr
    if new_arr.ndim == 1:
        new_arr = arr[:, None]
    return new_arr

def visualize_loss(loss_history):
    plt.plot(loss_history)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.show()

def visualize_preds(model):
    # Grid over (x1, x2)
    x1 = np.linspace(-1, 1, 50)
    x2 = np.linspace(-1, 1, 50)
    X1, X2 = np.meshgrid(x1, x2)
    X_grid = np.column_stack((X1.ravel(), X2.ravel()))

    # Model prediction on the grid
    y_pred = model.forward(X_grid).reshape(X1.shape)

    # True function for comparison
    y_true = (X1**2 + X2**2)

    fig = plt.figure(figsize=(12, 5))

    ax1 = fig.add_subplot(1, 2, 1, projection="3d")
    ax1.plot_surface(X1, X2, y_true)
    ax1.set_title(r"True Function: $x_1^2 + x_2^2$")
    ax1.set_xlabel(r"$x_1$")
    ax1.set_ylabel(r"$x_2$")
    ax1.set_zlabel(r"$y$")

    ax2 = fig.add_subplot(1, 2, 2, projection="3d")
    ax2.plot_surface(X1, X2, y_pred)
    ax2.set_title("Model Prediction")
    ax2.set_xlabel(r"$x_1$")
    ax2.set_ylabel(r"$x_2$")
    ax2.set_zlabel("Prediction")

    plt.tight_layout()
    plt.show()

def get_accuracy(net, X, y):
    y_pred = net.forward(X)
    y_pred_bin = np.where(y_pred > 0.5, 1, 0)
    return np.sum(y_pred_bin == y) / len(y)


## Activation functions
Activation functions add non-linearity. We'll use:
- **Identity** (useful for output layers in regression)
- **Sigmoid** (classic for binary classification demos)


In [ ]:
class Identity:
    def forward(self, x):
        return x

    def backward(self, x):
        return np.ones_like(x)

class Sigmoid:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def backward(self, x):
        f = self.forward(x)
        return f * (1 - f)


## Loss function (MSE)
Mean Squared Error is common for regression and simple experiments:
\begin{align}
\text{MSE}(y,\hat{y})=\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^2
\end{align}


In [ ]:
class MSE:
    def forward(self, y_pred, y):
        return np.mean((y_pred - y) ** 2)

    def backward(self, y_pred, y):
        return 2 * (y_pred - y)


## Core building blocks: Neuron, Layer, Network

- A **Neuron** computes: `z = x·w + b`, then applies an activation function.
- A **Layer** is a collection of neurons working in parallel.
- A **NeuralNetwork** is a stack of layers.


In [ ]:
class Neuron:
    def __init__(self, n_inputs, activation=Identity()):
        self.activation = activation
        self.w = 0.1 * np.random.randn(n_inputs).reshape(n_inputs, -1)
        self.b = 1.0
        self.z = None
        self.db = None
        self.dw = None

    def forward(self, x):
        # Linear part + activation
        self.z = np.dot(x, self.w) + self.b
        a = self.activation.forward(self.z)
        return a

    def backward(self, da, a_prev):
        da = add_dimension_optional(da)
        dz = da * self.activation.backward(self.z)

        # Gradients
        self.dw = np.dot(a_prev.T, dz)
        self.db = np.sum(dz, axis=0, keepdims=True)

        # Gradient for previous layer
        da_prev = np.dot(dz, self.w.T)
        return da_prev

    def step(self, l_r):
        # Gradient descent update
        self.w -= l_r * self.dw
        self.b -= l_r * self.db

class Layer:
    def __init__(self, n_inputs, n_neurons, activation=Identity()):
        self.neurons = [Neuron(n_inputs, activation) for _ in range(n_neurons)]
        self.A = None  # layer activations

    def forward(self, x):
        self.A = np.column_stack([n.forward(x) for n in self.neurons])
        return self.A

    def backward(self, da, a_prev):
        da_prev = np.array([n.backward(da[:, i], a_prev) for i, n in enumerate(self.neurons)])
        da_prev = np.sum(da_prev, axis=0)
        return da_prev

    def step(self, l_r):
        for n in self.neurons:
            n.step(l_r)

class NeuralNetwork:
    def __init__(self, layer_sizes, activation=Identity()):
        self.layers = []
        self._init_layers(layer_sizes, activation)

    def _init_layers(self, layer_sizes, activation):
        for n_in, n_out in zip(layer_sizes[:-1], layer_sizes[1:]):
            self.layers.append(Layer(n_in, n_out, activation))

    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer.forward(out)
        return out

    def backward(self, X, da):
        grad = da
        for i, layer in reversed(list(enumerate(self.layers))):
            a_prev = X if i == 0 else self.layers[i - 1].A
            grad = layer.backward(grad, a_prev)
        return grad

    def step(self, l_r):
        for layer in self.layers:
            layer.step(l_r)

    def predict(self, x):
        return self.forward(x)


## Training loop
A minimal supervised learning loop:
1) forward pass → predictions  
2) loss evaluation  
3) backward pass → gradients  
4) parameter update


In [ ]:
def train(net, X, y, epochs, l_r, l_f):
    loss_history = []
    for _ in range(epochs):
        y_pred = net.forward(X)
        loss = l_f.forward(y_pred, y)
        loss_history.append(loss)

        da = l_f.backward(y_pred, y)
        net.backward(X, da)
        net.step(l_r)

    return loss_history


## Demo 1: Learn AND (binary classification)


In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [0], [0], [1]])

net = NeuralNetwork([2, 1], activation=Sigmoid())

print("Before training")
print("Predictions:")
print(net.forward(X))
print(f"Accuracy: {100*get_accuracy(net, X, y):.1f}%")

loss_history = train(net, X, y, epochs=1000, l_r=0.5, l_f=MSE())

print("\nAfter training")
print("Predictions:")
print(net.forward(X))
print(f"Accuracy: {100*get_accuracy(net, X, y):.1f}%")

visualize_loss(loss_history)


## Demo 2: Learn XOR (requires a hidden layer)
XOR is not linearly separable, so we use a small network with a hidden layer.


In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

net = NeuralNetwork([2, 2, 1], activation=Sigmoid())
loss_history = train(net, X, y, epochs=10000, l_r=0.1, l_f=MSE())

print("Final predictions (XOR):")
print(net.predict(X))
print(f"Accuracy: {100*get_accuracy(net, X, y):.1f}%")

visualize_loss(loss_history)


## Demo 3: Approximate a smooth function
We fit:
\begin{align}
F(x_1, x_2) = x_1^2 + x_2^2
\end{align}
on random samples in [-1, 1].


In [ ]:
# Generate a small training set
X = np.array([[np.random.uniform(-1, 1), np.random.uniform(-1, 1)] for _ in range(30)])
y = np.array([[x[0]**2 + x[1]**2] for x in X])

net = NeuralNetwork([2, 10, 1], activation=Sigmoid())
loss_history = train(net, X, y, epochs=5000, l_r=0.01, l_f=MSE())

print("A few examples (input → prediction vs true):")
for i in range(5):
    x_i = X[i].reshape(1, 2)
    y_pred_i = net.predict(x_i)[0, 0]
    print(f"{x_i.flatten()} → {y_pred_i:.4f}  vs  {y[i,0]:.4f}")

visualize_loss(loss_history)


## Visual check (3D surface)
This compares the true function surface against the learned approximation on a grid.
If your surface looks off, re-run training (random initialization can change convergence).


In [ ]:
visualize_preds(net)
